# Imports

In [1]:
from utils.gui import interactive_segmentation_with_config

In [ ]:
interactive_segmentation_with_config()

Using processing mode from config: ramified
Loaded configuration from checkpoint at step 2
Loaded stack with shape (192, 1536, 1536)
Loaded initial segmentation from checkpoint with 85 objects
Loaded refined ROIs from checkpoint
Loaded saved configuration parameters


Calculating ramified features...
Found 85 unique masks. Starting processing with 23 parallel jobs.
Processing in batches of 1 masks.
Extracting surface points for all masks...
Extracted surfaces for masks 1-1 of 85
Extracted surfaces for masks 2-2 of 85
Extracted surfaces for masks 3-3 of 85
Extracted surfaces for masks 4-4 of 85
Extracted surfaces for masks 5-5 of 85
Extracted surfaces for masks 6-6 of 85
Extracted surfaces for masks 7-7 of 85
Extracted surfaces for masks 8-8 of 85
Extracted surfaces for masks 9-9 of 85
Extracted surfaces for masks 10-10 of 85
Extracted surfaces for masks 11-11 of 85
Extracted surfaces for masks 12-12 of 85
Extracted surfaces for masks 13-13 of 85
Extracted surfaces for masks 14-14 of 85
Extracted surfaces for masks 15-15 of 85
Extracted surfaces for masks 16-16 of 85
Extracted surfaces for masks 17-17 of 85
Extracted surfaces for masks 18-18 of 85
Extracted surfaces for masks 19-19 of 85
Extracted surfaces for masks 20-20 of 85
Extracted surfaces for

100%|██████████| 85/85 [03:00<00:00,  2.12s/it]


Calculating ramification metrics...


100%|██████████| 85/85 [10:23<00:00,  7.34s/it]


...

Applying equivalences to create consistent labeling...


Second-pass relabeling: 100%|██████████| 4/4 [00:01<00:00,  2.32it/s]


Relabeling for consecutive indices...


Applying new labels: 100%|██████████| 4/4 [00:01<00:00,  2.27it/s]


Found 93 connected components after stitching
Upsampling segmentation to original size...
Final memory usage: 17202.58 MB
Saved updated configuration to /home/kirill/Desktop/For_Kirill/Iba1_Morpho_BB_Blind (Copy)/A/A_processed_ramified/processing_config_ramified.yaml


In [1]:
import os
import pandas as pd

def organize_processing_dir(drctry, mode):
    """
    A function to organize the directory with multiple samples for processing with 3D segmentation
    Inputs:
    - drctry: the directory containing tif files and a single csv file with the mapping of x,y,z dimensions to the corresponding tif files; csv file should have columns: 
    Filename, Width (um), Height (um), Slices, Depth (um)

    -mode: 'nuclear' or 'ramified', determines which config template to use
    """
    mode= 'ramified'
    #Make a list of all the tif files in the directory
    tif_files = [f for f in os.listdir(drctry) if f.endswith('.tif')]
    #Make a list of all the csv files in the directory
    csv_files = [f for f in os.listdir(drctry) if f.endswith('.csv')]
    #Check that there is only one csv file in the directory
    if len(csv_files) != 1:
        raise ValueError('There should be only one csv file in the directory')
    #Load the csv file
    csv_file = csv_files[0]
    csv_path = os.path.join(drctry, csv_file)
    df = pd.read_csv(csv_path)
    #Check that the csv file has the correct columns
    if not all([col in df.columns for col in ['Filename', 'Width (um)', 'Height (um)', 'Slices', 'Depth (um)']]):
        raise ValueError('The csv file should have columns: Filename, Width (um), Height (um), Slices, Depth (um)')
    #Check that the csv file has the correct number of rows
    if len(df) != len(tif_files):
        print(len(df), len(tif_files))
        print(tif_files)
        raise ValueError('The csv file should have the same number of rows as the number of tif files in the directory')
    #Create a new directory for each tif file
    root_names = [f.split('.')[0] for f in tif_files]
    for root_name in root_names:
        new_dir = os.path.join(drctry, root_name)
        os.makedirs(new_dir, exist_ok=True)
        #Move the tif file to the new directory
        tif_file = root_name + '.tif'
        tif_path = os.path.join(drctry, tif_file)
        new_tif_path = os.path.join(new_dir, tif_file)
        os.rename(tif_path, new_tif_path)
        
        #Copy the correct yaml config file to the new directory
        if mode == 'nuclear':
            config_file = 'nuclear_config.yaml'
        elif mode == 'ramified':
            config_file = 'ramified_config.yaml'
        #Template config is in the same directory as this script
        config_path = config_file
        new_config_path = os.path.join(new_dir, config_file)
        #Copy using python's functionality rather than shell comands to avoid issues due to spaces or special characters in the file path
        with open(config_path, 'r') as f:
            with open(new_config_path, 'w') as f2:
                f2.write(f.read())

        #Populate the yaml file with the correct dimensions
        """
        The yaml file has the following field:
        voxel_dimensions:
            x: 471.4
            y: 471.4
            z: 1

        We need to populate the x, y, and z fields with the correct values from the csv file
        """
        row = df[df['Filename'] == root_name]
        x = row['Width (um)'].values[0]
        y = row['Height (um)'].values[0]
        z = row['Depth (um)'].values[0]
        #Read the yaml file
        with open(new_config_path, 'r') as f:
            lines = f.readlines()
        #Identify the line number where the voxel_dimensions field starts
        for i, line in enumerate(lines):
            if 'voxel_dimensions' in line:
                start = i
                break
        #Populate the x, y, and z fields
        lines[start+1] = '  x: {}\n'.format(x)
        lines[start+2] = '  y: {}\n'.format(y)
        lines[start+3] = '  z: {}\n'.format(z)
        #Write the new yaml file
        with open(new_config_path, 'w') as f:
            f.writelines(lines)
    


In [2]:
drctry = '/home/kirill/Desktop/For_Kirill/Iba1_Morpho_BB_Blind (Copy)'
organize_processing_dir(drctry, 'ramified')